In [18]:
import os
import pandas as pd
import numpy as np
import psycopg2
import psycopg2.extras as extras
import platform

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

conn = psycopg2.connect(
    database="graphyfinancials",
    host="localhost",
    user="derek",
    password="",
    port="5432"
)

cursor = conn.cursor()

In [19]:
def insert_into_db(conn, df, table:str):
    tuples = [tuple(x) for x in df.to_numpy()]
    cols = ','.join(list(df.columns))
    query = "INSERT INTO %s(%s) VALUES %%s " % (table,cols)
    cursor = conn.cursor()
    try:
        print('Starting insertion')
        extras.execute_values(cursor, query, tuples)
        conn.commit()
        print(f'Columns inserted:{cols}')
        print("the dataframe is inserted")
    except (Exception, psycopg2.DatabaseError) as error:
        print("error", error)
        conn.rollback()
        raise
    finally:
        cursor.close()
    cursor.close()

In [39]:

mac_data_directory = '/Users/derek/Documents/Github_Repos/graphy_financials/data'
# win_data_directory = '\Users\derek\Documents\Github_Repos\graphy_financials\data'

tsv_disclosure = []
tsv_coissuer_info = []
tsv_issuer_info = []
tsv_issuer_jurisdiction = []
tsv_issuer_signature = []
tsv_signature = []
tsv_submission = []

# def tsv_processor(arr_tsv: object):
#     for folder_name in 

for folder_name in os.listdir(mac_data_directory):
    folder_path = os.path.join(mac_data_directory, folder_name)
    
    if os.path.isdir(folder_path):
        for root, dirs, files in os.walk(folder_path):
            for file_name in files:
                if file_name.endswith('DISCLOSURE.tsv'):
                    tsv_path = os.path.join(root, file_name)
                    tsv_disclosure.append(tsv_path)
                if file_name.endswith('C_COISSUER_INFORMATION.tsv'):
                    tsv_path = os.path.join(root, file_name)
                    tsv_coissuer_info.append(tsv_path)
                if file_name.endswith('C_ISSUER_INFORMATION.tsv'):
                    tsv_path = os.path.join(root, file_name)
                    tsv_issuer_info.append(tsv_path)
                if file_name.endswith('ISSUER_JURISDICTIONS.tsv'):
                    tsv_path = os.path.join(root, file_name)
                    tsv_issuer_jurisdiction.append(tsv_path)
                if file_name.endswith('ISSUER_SIGNATURE.tsv'):
                    tsv_path = os.path.join(root, file_name)
                    tsv_issuer_signature.append(tsv_path)
                if file_name.endswith('C_SIGNATURE.tsv'):
                    tsv_path = os.path.join(root, file_name)
                    tsv_signature.append(tsv_path)
                if file_name.endswith('SUBMISSION.tsv'):
                    tsv_path = os.path.join(root, file_name)
                    tsv_submission.append(tsv_path)

In [40]:
def tsv_to_dataframe(tsv_list: object, multi_df=False):
    if multi_df == True:
        consolidated_df = []
        for tsv_file in tsv_list:
            df = pd.read_csv(tsv_file, sep='\t')
            consolidated_df.append(df)
        # print(consolidated_df)
        return pd.concat(consolidated_df, ignore_index=True)
    elif multi_df == False:
        df = pd.read_csv(tsv_list)
        return df

disclosure_dataframe = tsv_to_dataframe(tsv_disclosure, multi_df=True)
coissuer_info_dataframe = tsv_to_dataframe(tsv_coissuer_info, multi_df=True)
issuer_info_dataframe = tsv_to_dataframe(tsv_issuer_info, multi_df=True)
issuer_jurisdiction_dataframe = tsv_to_dataframe(tsv_issuer_jurisdiction, multi_df=True)
issuer_signature_dataframe = tsv_to_dataframe(tsv_issuer_signature, multi_df=True)
signature_dataframe = tsv_to_dataframe(tsv_signature, multi_df=True)
submission_dataframes = tsv_to_dataframe(tsv_submission, multi_df=True)

In [43]:
# COMPANY INFORMATION

df1 = issuer_info_dataframe
df2 = submission_dataframes
df = pd.merge(df1, df2, how='inner', on='ACCESSION_NUMBER')

columns_to_keep = ['NAMEOFISSUER', 'STREET1', 'STREET2', 'CITY', 'STATEORCOUNTRY', 'ZIPCODE', 'ISSUERWEBSITE', 'CIK', 'DATEINCORPORATION']

df = df[columns_to_keep]
df = df.replace(np.nan,None)
df = df.drop_duplicates(subset=['CIK'], keep='first')

df = df.rename(columns = {
    "NAMEOFISSUER": "company_title",
    "STREET1": "street_1",
    "STREET2": "street_2",
    "CITY": "city",
    "STATEORCOUNTRY": "state_or_country",
    "ZIPCODE": "zipcode",
    "ISSUERWEBSITE": "website",
    "CIK": "cik",
    "DATEINCORPORATION": "date_incorporation"
})
df.info()
df['cik'] = df['cik'].astype(str).str.replace(r'\..*','',regex=True)
# insert_into_db(conn, df,'companies')
company_df = df

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6215 entries, 0 to 23467
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   company_title       6214 non-null   object
 1   street_1            5795 non-null   object
 2   street_2            1716 non-null   object
 3   city                5795 non-null   object
 4   state_or_country    5795 non-null   object
 5   zipcode             5795 non-null   object
 6   website             5795 non-null   object
 7   cik                 6214 non-null   object
 8   date_incorporation  5795 non-null   object
dtypes: object(9)
memory usage: 485.5+ KB


In [46]:
# DISCLOSURES

disclosure_dataframe.rename(columns = {
    'ACCESSION_NUMBER': 'accession_number',
    'COMPENSATIONAMOUNT': 'compensation_amount_description', 
    'FINANCIALINTEREST': 'financial_interest',
    'SECURITYOFFEREDTYPE': 'security_offered_type',
    'SECURITYOFFEREDOTHERDESC': 'security_offered_other_desc',
    'NOOFSECURITYOFFERED': 'no_of_security_offered',
    'PRICE': 'price',
    'PRICEDETERMINATIONMETHOD': 'price_determination_method', 
    'OFFERINGAMOUNT': 'offering_amount',
    'OVERSUBSCRIPTIONACCEPTED': 'oversubscription_accepted',
    'OVERSUBSCRIPTIONALLOCATIONTYPE': 'oversubscription_allocation_type',
    'DESCOVERSUBSCRIPTION': 'desc_oversubscription',
    'MAXIMUMOFFERINGAMOUNT': 'maximum_offering_amount',
    'DEADLINEDATE': 'deadline_date',
    'CURRENTEMPLOYEES': 'current_employees',
    'TOTALASSETMOSTRECENTFISCALYEAR': 'total_assets_most_recent_fiscal_year',
    'TOTALASSETPRIORFISCALYEAR': 'total_assets_prior_fiscal_year',
    'CASHEQUIMOSTRECENTFISCALYEAR': 'cash_equity_most_recent_fiscal_year',
    'CASHEQUIPRIORFISCALYEAR': 'cash_equity_prior_fiscal_year',
    'ACTRECEIVEDRECENTFISCALYEAR': 'act_received_recent_fiscal_year', 
    'ACTRECEIVEDPRIORFISCALYEAR': 'act_received_prior_fiscal_year',
    'SHORTTERMDEBTMRECENTFISCALYEAR': 'short_term_debt_recent_fiscal_year',
    'SHORTTERMDEBTPRIORFISCALYEAR': 'short_term_debt_prior_fiscal_year',
    'LONGTERMDEBTRECENTFISCALYEAR': 'long_term_debt_recent_fiscal_year',
    'LONGTERMDEBTPRIORFISCALYEAR': 'long_term_debt_prior_fiscal_year',
    'REVENUEMOSTRECENTFISCALYEAR': 'revenue_most_recent_fiscal_year',
    'REVENUEPRIORFISCALYEAR': 'revenue_prior_fiscal_year', 
    'COSTGOODSSOLDRECENTFISCALYEAR': 'cost_goods_sold_recent_fiscal_year',
    'COSTGOODSSOLDPRIORFISCALYEAR': 'cost_goods_sold_prior_fiscal_year',
    'TAXPAIDMOSTRECENTFISCALYEAR': 'tax_paid_most_recent_fiscal_year',
    'TAXPAIDPRIORFISCALYEAR': 'tax_paid_prior_fiscal_year',
    'NETINCOMEMOSTRECENTFISCALYEAR': 'net_income_most_recent_fiscal_year',
    'NETINCOMEPRIORFISCALYEAR': 'net_income_prior_fiscal_year'
}, inplace = True )

# change data type
disclosure_dataframe = disclosure_dataframe.astype({
    'accession_number': 'object',
    'compensation_amount_description': 'object',
    'financial_interest': 'object',
    'security_offered_type': 'object',
    'security_offered_other_desc': 'object',
    'no_of_security_offered': 'float',
    'price': 'float',
    'price_determination_method': 'object',
    'offering_amount': 'float64',
    'oversubscription_accepted': 'object',
    'oversubscription_allocation_type': 'object',
    'desc_oversubscription': 'object',
    'maximum_offering_amount': 'object',
    'deadline_date': 'object',
    'current_employees': 'object',
    'total_assets_most_recent_fiscal_year': 'float',
    'total_assets_prior_fiscal_year': 'float',
    'cash_equity_most_recent_fiscal_year': 'float',
    'cash_equity_prior_fiscal_year': 'float',
    'act_received_recent_fiscal_year': 'float',
    'act_received_prior_fiscal_year': 'float',
    'short_term_debt_recent_fiscal_year': 'float',
    'short_term_debt_prior_fiscal_year': 'float',
    'long_term_debt_recent_fiscal_year': 'float',
    'long_term_debt_prior_fiscal_year': 'float',
    'revenue_most_recent_fiscal_year': 'float',
    'revenue_prior_fiscal_year': 'float', 
    'cost_goods_sold_recent_fiscal_year': 'float',
    'cost_goods_sold_prior_fiscal_year': 'float',
    'tax_paid_most_recent_fiscal_year': 'float',
    'tax_paid_prior_fiscal_year': 'float',
    'net_income_most_recent_fiscal_year': 'float',
    'net_income_prior_fiscal_year': 'float'
})

# disclosure_dataframe.drop('deadline_date',axis=1,inplace=True)
# disclosure_dataframe.info()
df = disclosure_dataframe.where(pd.notna(disclosure_dataframe), None)
df = disclosure_dataframe.replace(np.nan,None)

# df.to_csv('/Users/derek/Downloads/disclosure_extract.csv')

input_df = df[
        [
            'accession_number',
            'compensation_amount_description',
            'financial_interest',
            'security_offered_type',
            'security_offered_other_desc',
            'no_of_security_offered',
            'price',
            'price_determination_method',
            'offering_amount',
            'oversubscription_accepted',
            'oversubscription_allocation_type',
            'desc_oversubscription',
            'maximum_offering_amount',
            'deadline_date',
            'current_employees',
            'total_assets_most_recent_fiscal_year',
            'total_assets_prior_fiscal_year',
            'cash_equity_most_recent_fiscal_year',
            'cash_equity_prior_fiscal_year',
            'act_received_recent_fiscal_year',
            'act_received_prior_fiscal_year',
            'short_term_debt_recent_fiscal_year',
            'short_term_debt_prior_fiscal_year',
            'long_term_debt_recent_fiscal_year',
            'long_term_debt_prior_fiscal_year',
            'revenue_most_recent_fiscal_year',
            'revenue_prior_fiscal_year',
            'cost_goods_sold_recent_fiscal_year',
            'cost_goods_sold_prior_fiscal_year',
            'tax_paid_most_recent_fiscal_year',
            'tax_paid_prior_fiscal_year',
            'net_income_most_recent_fiscal_year',
            'net_income_prior_fiscal_year',
        ]
    ]

# input_df.to_csv('/Users/derek/Downloads/disclosure_extract_partial.csv')
input_df
# insert_into_db(conn, input_df,'temp_disclosures')


,accession_number,compensation_amount_description,financial_interest,security_offered_type,security_offered_other_desc,no_of_security_offered,price,price_determination_method,offering_amount,oversubscription_accepted,oversubscription_allocation_type,desc_oversubscription,maximum_offering_amount,deadline_date,current_employees,total_assets_most_recent_fiscal_year,total_assets_prior_fiscal_year,cash_equity_most_recent_fiscal_year,cash_equity_prior_fiscal_year,act_received_recent_fiscal_year,act_received_prior_fiscal_year,short_term_debt_recent_fiscal_year,short_term_debt_prior_fiscal_year,long_term_debt_recent_fiscal_year,long_term_debt_prior_fiscal_year,revenue_most_recent_fiscal_year,revenue_prior_fiscal_year,cost_goods_sold_recent_fiscal_year,cost_goods_sold_prior_fiscal_year,tax_paid_most_recent_fiscal_year,tax_paid_prior_fiscal_year,net_income_most_recent_fiscal_year,net_income_prior_fiscal_year
0,0001820003-20-000004,5.5% of the amount raised,None,Common Stock,None,10000.0,1.0,None,10000.0,Y,Pro-rata basis,None,245000.0,2020-12-31,3.0,0.0,0.0,0.0,122788.0,0.0,0.0,0.0,109914.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-55126.0,-262805.0
1,0001665160-20-001227,6.0 percent,Two percent (2%) of securities of the total am...,Common Stock,None,10869.0,0.92,None,9999.48,Y,Other,"At issuer's discretion, with priority given to...",249999.88,2021-03-29,4.0,89051.38,323701.85,9363.0,120164.08,11676.88,794.88,4307.0,0.0,1511247.57,1108913.1,69128.35,4596.2,63997.0,2321.95,800.0,0.0,-636984.0,-397503.57
2,0001826174-20-000002,commission of 7% on funds raised,Not applicable,Common Stock,None,10000.0,0.5,None,5000.0,Y,"First-come, first-served basis",None,107000.0,2020-11-15,0.0,21525.0,0.0,7257.0,0.0,3.0,0.0,0.0,0.0,7250.0,0.0,76.0,0.0,16.0,0.0,0.0,0.0,60.0,0.0
3,0001669191-20-000558,Up to 4.9% of amount raised for a successful o...,None.,Common Stock,None,20000.0,0.5,At the issuer's discretion.,10000.0,Y,"First-come, first-served basis",None,250000.0,2020-10-26,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0001669191-20-000556,Up to 4.9% of amount raised for a successful o...,None.,Common Stock,None,3216.0,3.11,At the issuer's discretion.,10001.76,Y,"First-come, first-served basis",None,444608.71,2020-10-30,2.0,45194.0,70501.0,29694.0,70501.0,15500.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-183454.0,-61790.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23469,0001907870-22-000002,"At the conclusion of the offering, the issuer ...",The Intermediary will also receive compensatio...,Other,Crowd SAFE (Simple Agreement for Future Equity),25000.0,1.0,Arbitrary.,25000.0,Y,Other,At the Intermediary's discretion.,5000000.0,2022-04-27,3.0,1000.0,0.0,1000.0,0.0,0.0,0.0,9575.0,0.0,0.0,0.0,0.0,0.0,8575.0,0.0,0.0,0.0,-8575.0,0.0
23470,0001665160-22-000600,7 - 13 percent,Two percent (2%) of securities of the total am...,Common Stock,None,125000.0,0.08,None,10000.0,Y,Other,"At issuer's discretion, with priority given to...",1070000.0,2022-03-17,8.0,17376.0,2184.0,10400.0,2184.0,0.0,0.0,49977.0,27248.0,0.0,0.0,8553.0,0.0,3338.46,0.0,0.0,0.0,-44536.0,-233820.0
23471,0001894171-22-000003,5.0%,2% of Raise Amount in SAFE Equity,Preferred Stock,None,167.0,300.0,None,50000.0,Y,Other,Management discretion,250000.0,2022-04-15,3.0,20000.0,0.0,5000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23472,0001917773-22-000002,6.0% of the Offering amount upon the successfu...,1.5% of securities sold from a successful Offe...,Other,SAFE NOTE,25000.0,1.0,Total of the Target Offering Amount divided by...,25000.0,Y,"First-come, first-served basis",None,1000000.0,2022-10-30,3.0,170064.0,0.0,25364.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
# ISSUER INFO

issuer_info_dataframe.rename(columns={
    'ACCESSION_NUMBER': 'accession_number',
    'ISAMENDMENT': 'is_amendment', 
    'PROGRESSUPDATE': 'progress_update',
    'NATUREOFAMENDMENT': 'nature_of_amendment',
    'NAMEOFISSUER': 'name_of_issuer',
    'LEGALSTATUSFORM': 'legal_status_form',
    'LEGALSTATUSOTHERDESC': 'legal_status_other_desc',
    'JURISDICTIONORGANIZATION': 'jurisdiction_organization',
    'DATEINCORPORATION': 'date_incorporation',
    'STREET1': 'street1',
    'STREET2': 'street2',
    'CITY': 'city',
    'STATEORCOUNTRY': 'state_or_country',
    'ZIPCODE': 'zipcode',
    'ISSUERWEBSITE': 'issuer_website',
    'COMPANYNAME': 'intermediary_name',
    'COMMISSIONCIK': 'intermediary_cik',
    'COMMISSIONFILENUMBER': 'commission_file_number',
    'CRDNUMBER': 'crd_number',
    'ISCOISSUER': 'is_co_issuer'
},inplace=True)

# issuer_info_dataframe.info()
df = issuer_info_dataframe.astype({
    'intermediary_cik':'string'
})


# data cleaning
df['date_incorporation'].fillna('1900-01-01',inplace=True)
df['intermediary_cik'] = df['intermediary_cik'].str.replace(r'\..*','',regex=True)
df['intermediary_cik'].fillna('',inplace=True)

df = df.where(pd.notna(df),None)

df.info()
df.head()


# df.to_csv('/Users/derek/Downloads/issuer_info_df.csv')


insert_into_db(conn, df, 'temp_issuer_info')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23474 entries, 0 to 23473
Data columns (total 20 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   accession_number           23474 non-null  object 
 1   is_amendment               23474 non-null  int64  
 2   progress_update            3575 non-null   object 
 3   nature_of_amendment        7862 non-null   object 
 4   name_of_issuer             23472 non-null  object 
 5   legal_status_form          22587 non-null  object 
 6   legal_status_other_desc    262 non-null    object 
 7   jurisdiction_organization  22587 non-null  object 
 8   date_incorporation         23474 non-null  object 
 9   street1                    22586 non-null  object 
 10  street2                    7121 non-null   object 
 11  city                       22587 non-null  object 
 12  state_or_country           22587 non-null  object 
 13  zipcode                    22587 non-null  obj

In [49]:
# submission_dataframes.info()
# submission_dataframes.head()

submission_dataframes.rename(columns={
    'ACCESSION_NUMBER':'accession_number',
    'SUBMISSION_TYPE':'submission_type',
    'FILING_DATE':'filing_date',
    'CIK':'cik',
    'FILE_NUMBER':'file_number',
    'PERIOD':'period'
},inplace=True)

filtered_df = submission_dataframes[['accession_number','submission_type','cik','file_number','filing_date','period']]

filtered_df['filing_date'] = pd.to_datetime(filtered_df['filing_date'], format='%Y%m%d')
filtered_df['period'] = pd.to_datetime(filtered_df['period'], format='%Y%m%d')
filtered_df['period'].fillna('1900-01-01',inplace=True)

df = filtered_df.astype({
    'cik':'string',
    'filing_date':'object',
    'period':'object'
})

df['cik'] = df['cik'].str.replace(r'\..*','',regex=True)
df['cik'].fillna('',inplace=True)


df.head(100)

# insert_into_db(conn,df,'temp_submission')


,accession_number,submission_type,cik,file_number,filing_date,period
0,0001820003-20-000004,C/A,1820003,020-26982,2020-09-30 00:00:00,1900-01-01 00:00:00
1,0001665160-20-001227,C,1823056,020-26995,2020-09-30 00:00:00,1900-01-01 00:00:00
2,0001826174-20-000002,C,1826174,020-26994,2020-09-30 00:00:00,1900-01-01 00:00:00
3,0001669191-20-000558,C/A,1822208,020-26833,2020-09-30 00:00:00,1900-01-01 00:00:00
4,0001669191-20-000556,C/A,1797609,020-26652,2020-09-30 00:00:00,1900-01-01 00:00:00
5,0001815319-20-000002,C,1815319,020-26993,2020-09-30 00:00:00,1900-01-01 00:00:00
6,0001665160-20-001226,C/A,1820834,020-26992,2020-09-30 00:00:00,1900-01-01 00:00:00
7,0001665160-20-001225,C/A,1756447,020-26584,2020-09-30 00:00:00,1900-01-01 00:00:00
8,0001665160-20-001223,C,1820834,020-26992,2020-09-30 00:00:00,1900-01-01 00:00:00
9,0001665160-20-001221,C/A,1818192,020-26653,2020-09-30 00:00:00,1900-01-01 00:00:00


In [24]:
# COISSUER

coissuer_info_dataframe.rename(columns={
    'ACCESSION_NUMBER':'accession_number',
    'ID':'id',
    'ISEDGARFILER':'is_edgar_filer',
    'COISSUERCIK':'co_issuer_cik',
    'NAMEOFCOISSUER':'name_of_co_issuer',
    'LEGALSTATUSFORM':'legal_status_form',
    'LEGALSTATUSOTHERDESC':'legal_status_other_desc',
    'JURISDICTIONORGANIZATION':'jurisdiction_organization',
    'DATEINCORPORATION':'date_incorporation',
    'STREET1':'street_1',
    'STREET2':'street_2',
    'CITY':'city',
    'STATEORCOUNTRY':'state_or_country',
    'ZIPCODE':'zipcode',
    'COISSUERWEBSITE':'co_issuer_website'
},inplace=True)

coissuer_info_dataframe['date_incorporation'] = pd.to_datetime(coissuer_info_dataframe['date_incorporation'], format='%d-%b-%Y')

df = coissuer_info_dataframe.astype({
    'accession_number':'object',
    'id':'object',
    'date_incorporation':'datetime64[ns]',
    'co_issuer_cik': 'object'
})

df = df.where(pd.notna(df), None)
# coissuer_info_dataframe

df = df.drop(columns=[
    'is_edgar_filer',
    'legal_status_form',
    'legal_status_other_desc',
    'jurisdiction_organization'
    ])

df = df.replace(np.nan, None)
# df.to_csv('/Users/derek/Downloads/coissuer_info_df.csv')
df

# insert_into_db(conn, coissuer_info_dataframe,'temp_coissuer_info')

,accession_number,id,co_issuer_cik,name_of_co_issuer,date_incorporation,street_1,street_2,city,state_or_country,zipcode,co_issuer_website
0,0001670254-21-000764,2,None,"BioSmart Research I EB, a series of Wefunder S...",2021-06-22,4104 24TH ST,PMB 8113,San Francisco,CA,94114,https://wefunder.com/
1,0001670254-21-000764,1,None,"BioSmart Research I, a series of Wefunder SPV,...",2021-06-22,4104 24TH ST,PMB 8113,San Francisco,CA,94114,https://wefunder.com/
2,0001670254-21-000762,2,None,"Invincible Entertainment Partners I EB, a seri...",2021-06-29,4104 24TH ST,PMB 8113,San Francisco,CA,94114,https://wefunder.com/
3,0001670254-21-000762,1,None,"Invincible Entertainment Partners I, a series ...",2021-06-29,4104 24TH ST,PMB 8113,San Francisco,CA,94114,https://wefunder.com/
4,0001670254-21-000760,1,None,"Nwicode I, a series of Wefunder SPV, LLC",2021-06-22,4104 24TH ST,PMB 8113,San Francisco,CA,94114,https://wefunder.com/
...,...,...,...,...,...,...,...,...,...,...,...
2117,0001670254-22-000277,2,None,"Go LoCo I EB, a series of Wefunder SPV, LLC",2022-04-01,4104 24TH ST,PMB 8113,San Francisco,CA,94114,https://wefunder.com/
2118,0001644600-22-000038,1,None,"Cosmic Shielding CF SPV, LLC",2021-07-09,395 Central Park Pl NE,Unit 340,Atlanta,GA,30312,www.cosmicshielding.com
2119,0001917773-22-000002,1,None,"SeaNSoul CF SPV, LLC",2022-02-23,1996 NEWELL RD,None,MALIBU,CA,90265,None
2120,0001670254-22-000274,1,None,"Boseco I, a series of Wefunder SPV, LLC",2021-12-16,4104 24TH ST,PMB 8113,San Francisco,CA,94114,https://wefunder.com/
